# Funções Auxiliares e Imports

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

file_path = "C:/Users/gabri/Downloads/Redes Complexas/"

## Lendo e plotando o grafo dos dados

### Base Hamsterster

In [ ]:
# Base de dados para a questão 1
ham = nx.read_edgelist(file_path + "data/hamsterster.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(ham)
# nx.draw(ham, pos, node_color="lightgray", node_size=500, with_labels=True)

### Base USairport500

In [ ]:
# Base de dados para a questão 2 e 4
usa = nx.read_edgelist(file_path + "data/USairport500.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(usa)
# nx.draw(usa, pos, node_color="lightgray", node_size=50, with_labels=True)

### Base Advogato

In [ ]:
# Base de dados para a questão 3
adv = nx.read_edgelist(file_path + "data/advogato.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(adv)
# nx.draw(adv, pos, node_color="lightgray", node_size=500, with_labels=True)

### Base Word_Adjacencies

In [ ]:
# Base de dados para as questões 4 e 5
word = nx.read_weighted_edgelist(file_path + "data/word_adjacencies.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(word)
# nx.draw(word, pos, node_color="lightgray", node_size=500, with_labels=True)

### Padronização da Rede

In [ ]:
# Função que "padroniza" a rede, transformando-a em uma rede sem direção, e retornando o maior componente conectado para a análise.
def padronizar_rede(G):
    # Transformamos o grafo em uma rede sem direção.
    G = G.to_undirected()
    G.remove_edges_from(nx.selfloop_edges(G))

    # Vamos selecionar apenas o maior componente conectado.
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G = G.subgraph(Gcc[0])

    # Transformando os labels para números inteiros, começando com 0:

    G = nx.convert_node_labels_to_integers(G, first_label=0)

    # Número de vértices e arestas:

    N = len(G)
    M = G.number_of_edges()
    print('Number of nodes:', N)
    print('Number of edges:', M)
    return G



## Funções da Questão 1

### Cálculo do Grau Médio

In [ ]:
# Função para calcular o grau médio do grafo.
def mean_degree(G):
    vk = dict(G.degree()).values()
    vk = np.array(list(vk))
    print('Degree', vk)



    md = np.mean(vk)
    # print('Mean degree: ', md)
    return md



### Cálculo da Distribuição do Grau

In [ ]:
# Função para calcular a distribuição do grau do grafo.
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values())  # we get only the degree values
    vk = np.array(vk)
    maxk = np.max(vk)
    mink = np.min(vk)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

# Função para calcular o momento da distribuição do grau do grafo.
def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

# Função para calcular a variância do grau.
def variance(G):
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    return variance
# print("Variância do grau = ", variance)

### Cálculo do Momento do Grau

In [ ]:
# Função para calcular o momento do grafo.
def momment(G,m):
    M = 0
    N = len(G)
    for i in G.nodes:
        M = M + G.degree(i)**m
    M = M/N
    return M

### Cálculo da Entropia de Shannon

In [ ]:
# Função para calcular a entropia de Shannon do grafo.
def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H

# Função para calcular a entropia de Shannon normalizada do grafo.
def normalized_shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H/np.math.log(len(G),2)

### Cálculo da Transitividade e do Coeficiente de Agrupamento

In [ ]:
# Função para calcular o coeficiente de transitividade do grafo.
def graph_transitivity(G):
    CC = (nx.transitivity(G))
    # print("Transitivity = ","%3.4f"%CC)
    return "%3.4f"%CC

# Função para calcular o coeficiente de agrupamento do grafo.
def avg_clustering_coefficient(G):
    avc = nx.average_clustering(G)
    # print("Average clustering:", "%3.4f"%avc)
    return "%3.4f"%avc

## Funções da Questão 2

### Distância

In [ ]:
# Função para calcular e plotar a distribuição de grau da rede.
def distance(G):
    if nx.is_connected(G) == True:
        l = nx.average_shortest_path_length(G)
        print("Average shortest path length:", "%3.4f"%l)
    else:
        print("The graph has more than one connected component")


    N = len(G)
    
    d = nx.diameter(G)
    print('Network diameter:', d)



    E = nx.global_efficiency(G)
    print('Network efficiency', E)

    leff = nx.local_efficiency(G)
    print('The average local efficiency of the network:', leff)



    if nx.is_connected(G) == True:
        D = np.zeros(shape=(N,N)) # D is the matrix of distances
        vl = []
        for i in np.arange(0,N):
            for j in np.arange(i+1, N):
                if(i != j):
                    aux = nx.shortest_path(G,i,j)
                    dij = len(aux)-1
                    D[i][j] = dij
                    D[j][i] = dij
                    vl.append(dij)
        x = range(0,d+1)
        plt.hist(vl, bins = x, density=True, color='#0504aa',alpha=0.7, rwidth=0.85)
        plt.title("Distribution of the geodesic distances", fontsize=20)
        plt.ylabel("P(l)", fontsize=15)
        plt.xlabel("Shortest path length (l)", fontsize=15)
        #plt.grid(True)
        plt.savefig('av_short_path.svg')
        plt.show(True)
    else:
        print("The graph has more than one connected component")



### Cálculo da Correlação de Grau a Grau

In [ ]:
# Função para calcular e plotar a correlação de grau a grau (degree-degree correlation).
def degree_correlation(G):
    r=nx.degree_assortativity_coefficient(G)
    print("Assortativity = ","%3.4f"%r)

    ki = []
    kj = []
    for i in range(0,len(G.nodes())):
        for j in range(0, len(G.nodes())):
            if(G.has_edge(i,j) == True):
                ki.append(G.degree(i))
                kj.append(G.degree(j))
                
    from scipy.stats import pearsonr
    # calculate Pearson's correlation
    corr, _ = pearsonr(ki, kj)
    print('Pearsons correlation: %.3f' % corr)



    knn = []
    for i in G.nodes():
        aux =  nx.average_neighbor_degree(G, nodes = [i])
        knn.append(float(aux[i]))
    knn = np.array(knn)
    print("Average degree of the neighborhood of the network:", "%3.2f"%np.mean(knn))



    vk = dict(G.degree())
    vk = list(vk.values())



    knnk = list()
    ks = list()
    for k in np.arange(np.min(vk), np.max(vk)+1):
        aux = vk == k
        if(len(knn[aux]) > 0):
            av_knn = np.mean(knn[aux]) #average clustering among all the nodes with degree k
            knnk.append(av_knn)
            ks.append(k)
    fig= plt.figure(figsize=(10,6))

    plt.plot(ks, knnk, '-o', color='gray',markersize=10, linewidth=0,
            markerfacecolor='lightgray',
            markeredgecolor='black',
            markeredgewidth=2)
    #plt.loglog(ks,knnk,'bo',basex=10,basey=10)
    #plt.title("Average neighborhood connectivity vs degree")
    plt.ylabel("knn(k)", Fontsize = 20)
    plt.xlabel("k", Fontsize = 20)
    #plt.savefig('knnk.eps')

    # determine best fit line
    par = np.polyfit(ks, knnk, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    xl = [min(ks), max(ks)]
    yl = [slope*xx + intercept  for xx in xl]
    plt.plot(xl, yl, '--', linewidth=3, color='black')
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.savefig('knn.eps') #save the figure into a file
    plt.show(True)



    rho = np.corrcoef(ks, knnk)[0,1]
    print('Pearson correlation coefficient:', rho)

    from scipy import stats
    s = stats.spearmanr(ks, knnk)
    print('Spearman rank correlation coefficient:', s)

### Caminhadas Aleatórias

In [ ]:
import random as random

edgelist=[(0,1),(1,2),(2,3), (3,1), (4,1), (4,5)]
G = nx.Graph(edgelist)
pos=nx.spring_layout(G)
nx.draw(G, with_labels = True, node_size=500, font_size=16, pos = pos)
plt.show(True)

# random walk of length T
T = 10
seed_node = 0
# neighbors
ng = G.neighbors(seed_node)
walk = []
for t in range(0,T):
    next_node = random.choice(list(ng))
    ng = G.neighbors(next_node)
    walk.append(next_node)
print('Walk:', walk)



nx.draw(G, with_labels = True, node_size=500, font_size=16, pos = pos)
plt.show(True)

A = nx.to_numpy_matrix(G)
print(A)
print('Number of walks of length two:')
print(A@A)



# Questões

## Questão 1)

Para a rede social Hamsterster, calcule a média dos menores caminhos e o diâmetro. Considere apenas o maior componente.

## Questão 2)

Para a rede de aeroportos (base USairport500), calcule a média e variância do comprimento dos menores caminhos. Considere apenas o maior componente na rede.

## Questão 3)

Calcule o coeficiente de assortatividade para a rede Advogato. Considere apenas o maior componente. Considere o valor mais próximo.

## Questão 4)

Para a rede de aeroportos (USairport500), calcule a entropia de Shannon considerando o comprimento dos menores caminhos. Use o logaritmo na base 2 e considere apenas o maior componente.

## Questão 5)

Calcule o coeficiente de correlação de Pearson entre o grau médio dos vizinhos e o grau para a rede de palavras (word_adjacencies). Considere apenas o maior componente conectado.